In [3]:
from functools import lru_cache, wraps
from itertools import chain
from contextlib import contextmanager, ContextDecorator
    
class State:
    """
    State is central Shoebot, whether it's keeping track of color or location.
    
    The State class provides a DSL to define and store state.
    Attributes that should be tracked can be added to the fields attribute.
    
    See the classes that extend State for examples, e.g. ColorState.
    """
    states = []
    
    def __init__(self, **kwargs):
        """
        Populate self.states as an aggregate of the states field in this
        class and its bases.
        
        Attributes in self.states that have not already been created are
        set to None.
        """
        self.states = [*chain.from_iterable(self._base_states())]
        
        for name in self.states:
            if not hasattr(self, name):
                setattr(self, name, kwargs.get(name))
    
    @classmethod
    def _base_state_classes(cls):
        return [base for base in cls.mro()
                if base is not State and issubclass(base, State)]
        
    @classmethod
    def _base_states(cls):
        """
        :return list of state names from any base classes that inherit from State
        """
        return [klass.states for klass in cls._base_state_classes()]
    
    def update(self, **kwargs):
        unknown_states = []
        for k, v in kwargs.items():
            if k in self.states:
                print("setattr ", k, v)
                setattr(self, k, v)
            else:
                unknown_states.append(k)
                
        if unknown_states:
            raise ValueError(f"Attempted to update unknown states")
    
    def as_dict(self, fields=None):
        if self.__class__ == State:
            return {}
        
        if fields:
            return {name: getattr(self, name) for name in fields if name in self.states}
        
        return dict(#super().as_dict(),
                    **{name: getattr(self, name) for name in self.states})
                              
    def dump(self):
        return ", ".join([f"{k}={v}"
                        for k, v in self.as_dict().items()])
                              
    def __repr__(self):
        #if hasattr(super(), "dump"):
        #return f"<{self.__class__.__name__} {super().dump()} {self.dump()}>"
        return f"<{self.__class__.__name__} {self.dump()}>"

    
class ColorStateMixin(State):    
    states = ["fill", "stroke", "blendmode"]
    
class ColorState(ColorStateMixin):
    def __init__(self):
        State.__init__(self)

class TransformStateMixin(State):
    states = ["origin"]
    
    def __init__(self):
        self.origin = (0, 0)
        State.__init__(self)
    
# TODO have state internal ?  ColorState, TranslationState etc ?
# class PathState(ColorState, TranslationState)
# self._state = PathState(....)
# self._state.fill = (1., 1., .5)
# self._state.blendmode = BlendMode.xor

class PathState(ColorStateMixin, TransformStateMixin):
    states = ["elements"]
    
    def __init__(self):
        self.elements = []
        self.fill = (1., 1., 1., 1.)
        self.stroke = (0., 0., 0., 1.)
        self.blendmode = "OVER"
        
        ColorStateMixin.__init__(self)
        TransformStateMixin.__init__(self)


class Path:
    """
    Path containing Bezier curves and lines.
    """
    def __init__(self): 
        self._state = PathState()
        
    def stroke(self, *args):
        if args:
            self._state.update(stroke=args)
        return self._state.stroke
    
    def fill(self, *args):
        if args:
            self._state.update(fill=args)
        return self._state.fill
    
    def append(self, element):
        self._state.append()
        
    def __repr__(self):
        return f"<Path {{{self._state.dump()}}}>"
    

# class StateField:
#     def __init__(self, state, field):
#         self.state = state
#         self.field = field
        
#     def get(self):
#         return self.state[self.field]

# class ColorState(State):
#     states = ["rgba", "restore_rgba"]
    
#     def __init__(self):
#         self.restore_rgba = False
#         State.__init__()
            
#     def restore(self):
#         if self.restore_color is False:
#             raise Exception()
#         self.rgba = self.restore_color
#         self
    

class SavedState:
    def __init__(self, state, fields):
        self.state = state
        self.saved = state.as_dict(fields)
        
    def restore(self):
        self.state.update(**self.saved)
        
class StateRef(State):
    # essentially a FK to fields stored in some other state
    def __init__(self, state, fields=None):
        if fields is None:
            fields = []
        self.states = states
        self.state = state

# class ColorState(StateRef):
#     states = ["rgba"]

#     def __init__(self, *args):
#         self.rgba = rgba
        
#     def dump(self):
#         return self.rgba
    
class Color:
    def __init__(self, rgba, state=None, save=None):
        if state is None:
            self._state = ColorState(rgba)
        else:
            self._state = state
            self._state.rgba = rgba
        self._saved_state = save
        
    @property
    def rgba(self):
        return self._state.rgba
    
    def __enter__(self):
        return self
    
    def __exit__(self, *args):
        self._saved_state.restore()
    
    def __repr__(self):
        return f"<Color {{{self._state.dump()}}}>"


class ContextState(ColorStateMixin):
    def __init__(self, **kwargs):
        ColorStateMixin.__init__(self, **kwargs)


class Context:
    def __init__(self):
        self._state = ContextState(fill=ColorState((.1, .1, .1)))
    
    def fill(self, *rgba):
        if rgba:
            color = Color(rgba, state=self._state, save=SavedState(self._state.fill, ["fill"]))
        else:
            color = Color(self._state.fill, state=self._state, save=SavedState(self._state, ["fill"]))
        return color

    def __repr__(self):
        return f"<Context {{{self._state.dump()}}}>"

        
    
ctx = Context()
print("start")
print(ctx)
print(ctx.fill())
print()

print("-----")
with ctx.fill(0, 0, 0):
    print("inside cm")
    print(ctx)
    print(ctx.fill())
    print()
    
print("-----")

    
print("end")
print(ctx)
print(ctx.fill())

ctx
#print(dir(p.fill))

TypeError: __init__() takes 1 positional argument but 2 were given

In [ ]:
rgba=StateRef(state, fill)